In [ ]:
import os
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')

from sklearn import metrics
import numpy as np
import pandas as pd
import random
import math


import cv2
from PIL import Image
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import timm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader,random_split

I thought it would be nice <br>
if there was an probability to be sure of their prediction.

In [ ]:
def dataframe(df, istrain=True):
    if istrain:
        mode = 'train'
    else:
        mode = 'test'
    df['filepath'] = df.id.apply(lambda x: '../input/seti-breakthrough-listen/'+ f'{mode}/'+str(x[0])+f'/{x}.npy')
    return df
test_df = pd.read_csv('../input/seti-breakthrough-listen/sample_submission.csv')
test_df = dataframe(test_df, istrain=False)
test_df['target'] = 0
train_df = pd.read_csv('../input/seti-breakthrough-listen/train_labels.csv')
train_df = dataframe(train_df)
train_df.head(2)

In [ ]:
class setiDataset(Dataset):
    def __init__(self, df, transform = None): 
        self.df = df
        self.transform = transform
    
    def fileinfo(self, idx):
        return self.df.filepath.iloc[idx], self.df.target.iloc[idx]
    
    def loadfile(self, filepath):
        image = np.load(filepath).astype('float32')
        return image
    
    def __getitem__(self, idx):
        filepath, target = self.fileinfo(idx)
        image = self.loadfile(filepath)
        return  torch.tensor(image, dtype=torch.float), torch.tensor(target, dtype=torch.long)
    
    def __len__(self):
        return len(self.df)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, padding = 1 if kernel_size == 3 else 0, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.vals = nn.Parameter(torch.zeros(out_channels)) 
        nn.init.kaiming_normal_(self.conv.weight, mode='fan_in', nonlinearity='leaky_relu')
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x += self.vals.view(1, self.bn.num_features, 1, 1).expand_as(x)
        x = x * torch.tanh(F.softplus(x))
        return x
    
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int):
        super().__init__()
        self.conv1 = ConvBlock(in_channels, out_channels, 3)
        self.conv2 = ConvBlock(out_channels, out_channels, 3)

    def forward(self, x):
        initial = x
        x = self.conv1(x)
        x = self.conv2(x)
        x = x + initial
        x = x * torch.tanh(F.softplus(x))
        return x

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, in_channels = 6, out_channels = 20, res_level = 2, pretrained=True):
        super(CustomModel,self).__init__()
        # residual layer
        self.conv_input = ConvBlock(in_channels, out_channels, 3)
        self.res_layer = nn.Sequential(
            *[ResBlock(out_channels, out_channels) for _ in range(res_level)]
        )
        self.conv_output = ConvBlock(out_channels, 2, 1)
        
        # prob layer
        self.prob_fc_1 = nn.Linear(2*273*256, 256)
        self.prob_fc_2 = nn.Linear(256, 1)
        
        self.model = timm.create_model('efficientnet_b0', pretrained=pretrained, in_chans=2, num_classes=1)

    def extract(self, x):
        x = self.conv_input(x)
        x = self.res_layer(x)
        x = self.conv_output(x)
        return x
    
    def forward(self, x):
        # residual layer
        x = self.extract(x)
        
        # prob layer
        prob = self.prob_fc_2(self.prob_fc_1(torch.flatten(x, start_dim=1)))
        prob = torch.sigmoid(prob)
        
        x = self.model(x)
        return prob, x

In [ ]:
def train(data_loader, model, optimizer, device):
    model.train()
    for data in tqdm(data_loader, position=0, leave=True, desc='Training'):
        images, targets = data
        images = images.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)
        def closure():
            optimizer.zero_grad()
            prob, output = model(images)
            loss1 = nn.BCEWithLogitsLoss(reduction='none')(output, targets.view(-1,1))
            loss2 = nn.BCEWithLogitsLoss(reduction='none')(-output, targets.view(-1,1))
            loss = (1-prob/2)*loss1 + (prob/2)*loss2
            loss = loss.mean()
            loss.backward()
            return loss
        optimizer.step(closure)

In [ ]:
def evaluate(data_loader, model, device):
    model.eval()
    
    final_targets = []
    final_outputs = []
    validate_losses = 0
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)
            
            prob, output = model(images)
            loss = nn.BCEWithLogitsLoss()(output, targets.view(-1, 1))
            validate_losses += loss.item()


            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()
            
            final_targets.extend(targets)
            final_outputs.extend(output)
    return final_outputs, final_targets, validate_losses/len(data_loader)

In [ ]:
def submission(data_loader, model, device):
    model.eval()
    
    final_outputs = []
    with torch.no_grad():
        
        for data in tqdm(data_loader, position=0, leave=True, desc='Evaluating'):
            images, targets = data

            images = images.to(device, dtype=torch.float)
            
            prob, output = model(images)

            output = output.detach().cpu().numpy().tolist()
            final_outputs.extend(output)
    return final_outputs

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device('cpu')
    print("GPU not available, CPU used")

In [ ]:
cmodel = CustomModel()
cmodel.to(device)

In [ ]:
Batch_Size = 16
init_dataset = setiDataset(train_df)
lengths = [int(len(init_dataset)*0.8), int(len(init_dataset)*0.2)]

train_dataset, valid_dataset = random_split(init_dataset, lengths)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_Size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=Batch_Size, shuffle=False)
optimizer=  torch.optim.Adam(cmodel.parameters(), lr=3e-4, eps=1e-5)

for epoch in range(10):
    print(epoch)
    train(train_loader, cmodel, optimizer, device)
    valid_pred, target, valid_loss = evaluate(valid_loader, cmodel, device)
    roc_auc = metrics.roc_auc_score(target, valid_pred)
    print(f"Epoch={epoch}, Valid Loss={valid_loss}, Valid ROC AUC={roc_auc}")

In [ ]:
valid_pred, target, valid_loss = evaluate(valid_loader, cmodel, device)
roc_auc = metrics.roc_auc_score(target, valid_pred)
print(f"Epoch={epoch}, Valid Loss={valid_loss}, Valid ROC AUC={roc_auc}")

too slow...

I just want to make below, but too slow.

original loss + prob/2 * (opposite loss - orignal loss)

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

test_dataset = setiDataset(test_df)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_Size, shuffle=False)
pred = submission(test_loader, cmodel, device)
sub = test_df[['id']].copy()
sub['target'] = sigmoid(np.array(pred).flatten())
sub.to_csv('submission.csv', index=None)